In [1]:
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
import tensorflow as tf
import numpy as np
if __name__ == "__main__":
    sys.path.append("../../")
from notebooks_src.metrics.mAP import calc_batch_metrics, EpochMetrics, calc_ap_one_class

importing Jupyter notebook from ../../notebooks_src/metrics/mAP.ipynb
importing Jupyter notebook from ../../notebooks_src/losses/util.ipynb
importing Jupyter notebook from ../../notebooks_src/box_encode_decode/configs.ipynb
importing Jupyter notebook from ../../notebooks_src/models/configs.ipynb
importing Jupyter notebook from ../../notebooks_src/config_util.ipynb
importing Jupyter notebook from ../../notebooks_src/load_data/configs.ipynb
importing Jupyter notebook from ../../notebooks_src/box_encode_decode/ssd/encode.ipynb
importing Jupyter notebook from ../../notebooks_src/box_encode_decode/ssd/make_anchors_orig.ipynb
importing Jupyter notebook from ../../notebooks_src/load_data/get_generator.ipynb
importing Jupyter notebook from ../../notebooks_src/load_data/generator/generator.ipynb
importing Jupyter notebook from ../../notebooks_src/box_encode_decode/util.ipynb
importing Jupyter notebook from ../../notebooks_src/load_data/generator/batch_fetcher.ipynb
importing Jupyter notebook fr

NameError: name 'final_metrics' is not defined

In [ ]:
def fit(model, generator, val_generator,num_epochs, loss_func, opt):
    with tf.Session() as sess:
        steps_per_epoch= generator.num_ims / generator.batch_size
        val_steps_per_epoch = val_generator.num_ims / val_generator.batch_size
        input_ = model.input
        loss_tensor, label  = get_loss_tensor(loss_func, model, generator)

        train_step = opt.minimize(loss_tensor)
        batch_accuracy_tensors, y_true = get_batch_accuracy_tensors(calc_batch_metrics, model, val_generator)
        
        
        sess.run(tf.global_variables_initializer())
  
        for ep in range(num_epochs):
            epm = EpochMetrics()
            for step in range(steps_per_epoch):
                im, boxes = generator.next()
                _,cur_loss = sess.run([train_step, loss_tensor],feed_dict={input_:im,label:boxes})
                
                print "loss: %8.4f"% cur_loss
            
            for step in range(val_steps_per_epoch):
                im, boxes = val_generator.next()
                updated_metrics = sess.run(batch_accuracy_tensors, feed_dict={y_true:boxes, input_:im})
                epm.update_metrics(*updated_metrics)
            
            final_metrics = epm.get_final_metrics()
            aps_voc12, placeholders= calc_ap_one_class()
            all_aps12 = {}
    
        
            for c in final_metrics[0].keys():
                placefillers = [d[c] for d in final_metrics]
                all_aps12[c] = sess.run(aps_voc12, feed_dict = dict(zip(placeholders, placefillers)) )
            

            print "AP's", all_aps12
            mAP12 = np.mean(all_aps12.values())
            print "accuracy: ", mAP12
                
                
                
                

                
#TODO: add streaming mAP calc for train and val

In [ ]:
def get_y_true_y_preds_tensors(model, batch_size, label_shape):
    output_tensors = model.outputs
        
    label_batch_shape = tuple([batch_size] + list(label_shape))
        
        
    label_tensor = tf.placeholder(tf.float32,shape=label_batch_shape, name="label")
    return label_tensor, output_tensors

In [ ]:
def get_loss_tensor(loss_func, model, generator):
        batch_size = generator.batch_size
        
        
        y_true, y_preds = get_y_true_y_preds_tensors(model, batch_size,generator.data.labels.shape[1:])
        loss_tensor = loss_func(y_true, y_preds)

        return loss_tensor, y_true

In [ ]:
def get_batch_accuracy_tensors(acc_func, model,generator):
        batch_size = generator.batch_size
        y_true, y_preds = get_y_true_y_preds_tensors(model, batch_size,generator.data.labels.shape[1:])
        batch_metrics = calc_batch_metrics(y_true, y_preds)
        return batch_metrics, y_true
        
    